<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install optuna
!pip install LAMDA-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 616.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.w

Importering av nødvendig bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev
import time

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection

# Modell
from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [ ]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import umarkert_andel_dag
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts_semi

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_semiveiledet.py /content/moduler_optimalisering_evaluering_semiveiledet.py

/content


In [ ]:
from moduler_optimalisering_evaluering_semiveiledet import BHD
from moduler_optimalisering_evaluering_semiveiledet import SelfTrainingRandomForestRegressor

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_enzym

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_dag

from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_1
from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_2

from moduler_optimalisering_evaluering_semiveiledet import bhd_andel_maks
from moduler_optimalisering_evaluering_semiveiledet import andel_umarkert

# Importering av relevant data

In [ ]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                   header=0,
                   sep=',',
                   index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                          header=0,
                          sep=',',
                          index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 2024

# Lagring av res

In [ ]:
# Oppretter instans for lagring av res
res_semi = Resultat_dicts_semi()

# Setter fast testandel på 20%, definerer treningsandeler og markerte andeler

In [ ]:
# Fast testsandel
test_andel = 0.2

# Treningsandeler til undersøkelse
treningsandeler = [0.25, 0.5, 0.75, 1.0]

# Markerte andeler til undersøkelse
andel_av_markert = [0.5, 1.0, 10.0]

# Parametervalg for hyperoptimalisering

In [ ]:
# Antall runder for hyperoptimalisering
n_runder_hel = 100
n_runder_andel = 100

# Antall gjentagelser
n_gjentagelser = 2

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Setter fast testandel og treningsandel
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [ ]:
# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt,
                         dict_res=res_semi.dict_res_1_hel,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)


# Lagring av resulatater
res_1_hel_semi = pd.DataFrame(res_semi.dict_res_1_hel)
res_1_hel_opt_semi = pd.DataFrame(res_semi.dict_res_1_hel_opt)

res_1_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_semi.csv')
res_1_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_opt_semi.csv')

[I 2024-04-30 18:46:00,219] A new study created in memory with name: no-name-0699b9dd-4a90-4df9-8dba-3d6c1e0adb1c
[I 2024-04-30 18:46:55,156] Trial 1 finished with value: 617.6555960481775 and parameters: {'k1': 10, 'k2': 9, 'p1': 10, 'p2': 4, 'max_iters': 1, 'pool_size': 37}. Best is trial 1 with value: 617.6555960481775.
[I 2024-04-30 18:58:32,156] Trial 2 finished with value: 587.5797280758849 and parameters: {'k1': 4, 'k2': 6, 'p1': 4, 'p2': 9, 'max_iters': 11, 'pool_size': 85}. Best is trial 2 with value: 587.5797280758849.
[I 2024-04-30 19:15:30,553] Trial 0 finished with value: 571.2047464726944 and parameters: {'k1': 1, 'k2': 8, 'p1': 9, 'p2': 10, 'max_iters': 26, 'pool_size': 98}. Best is trial 0 with value: 571.2047464726944.
[I 2024-04-30 19:22:54,255] Trial 3 finished with value: 570.6509095730211 and parameters: {'k1': 10, 'k2': 1, 'p1': 6, 'p2': 5, 'max_iters': 43, 'pool_size': 41}. Best is trial 3 with value: 570.6509095730211.
[I 2024-04-30 19:26:28,146] Trial 4 pruned.

Beste modell

## Alternativ 1: Forskjellige markerte og umarkerte treningsandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert, _, _,\
      _, _, _\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert

  # Går igjennom andeler av satt markerte andeler
  for andel in andel_av_markert:

    # Finner tilsvarende umarkerte andel
    umarkert_andel = andel_umarkert(andel, treningssett_markert, trening_umarkert)

    # Filtrer umarkert sett
    ubenyttet_andel = (1.0 - umarkert_andel)
    treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
    = train_test_split(trening_umarkert,
                       enzymtyper_umarkert,
                       test_size=ubenyttet_andel,
                       stratify=enzymtyper_umarkert,
                       random_state=random_seed)

    # Treningssett
    treningssett = pd.concat([treningssett_markert, treningssett_umarkert],
                            axis=0)
    treningssett = treningssett.sort_index(ascending=True)


    # Antall splitt etter minste enxymtype
    n_splitt = n_splitt_enzym(rå_data, treningssett_markert)

    # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
    semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt,
                             dict_res=res_semi.dict_res_1_hel,
                             rå_data=rå_data,
                             treningssett_markert=treningssett_markert,
                             enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                             treningssett_umarkert=treningssett_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                             treningssett=treningssett,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_andel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

    # Lagring av resulatater
    res_1_hel_semi = pd.DataFrame(res_semi.dict_res_1_hel)
    res_1_hel_opt_semi = pd.DataFrame(res_semi.dict_res_1_hel_opt)

    res_1_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_semi.csv')
    res_1_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_opt_semi.csv')

[I 2024-04-29 21:35:24,290] A new study created in memory with name: no-name-7c27ba89-610a-4478-8a39-f831ae3c4013
[I 2024-04-29 21:38:18,019] Trial 0 finished with value: 1136.7881609756505 and parameters: {'k1': 10, 'k2': 6, 'p1': 8, 'p2': 9, 'max_iters': 24, 'pool_size': 32}. Best is trial 0 with value: 1136.7881609756505.
[I 2024-04-29 21:38:28,524] Trial 1 finished with value: 1174.686404622968 and parameters: {'k1': 9, 'k2': 7, 'p1': 7, 'p2': 5, 'max_iters': 22, 'pool_size': 60}. Best is trial 0 with value: 1136.7881609756505.
[I 2024-04-29 21:40:42,984] Trial 2 finished with value: 891.5471735999575 and parameters: {'k1': 10, 'k2': 2, 'p1': 9, 'p2': 8, 'max_iters': 17, 'pool_size': 86}. Best is trial 2 with value: 891.5471735999575.
[I 2024-04-29 21:41:08,408] Trial 3 finished with value: 893.4284063532599 and parameters: {'k1': 7, 'k2': 1, 'p1': 5, 'p2': 1, 'max_iters': 32, 'pool_size': 74}. Best is trial 2 with value: 891.5471735999575.
[I 2024-04-29 21:42:15,445] Trial 4 finis

KeyboardInterrupt: 

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Definerer trening og testandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

In [ ]:
# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt,
                         dict_res=res_semi.dict_res_2_hel,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_2_hel_semi = pd.DataFrame(res_semi.dict_res_2_hel)
res_2_hel_opt_semi = pd.DataFrame(res_semi.dict_res_2_hel_opt)

res_2_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_hel_semi.csv')
res_2_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_hel_opt_semi.csv')

## Alternativ 2: Forskjellige markerte treningssandeler og umarkerte andeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, _, _\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       test_ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert

  # Går igjennom andeler av markerte andeler
  for andel in andel_av_markert:

    # Finner tilsvarende umarkerte andeler
    umarkert_andel = andel_umarkert(andel, treningssett_markert, trening_umarkert)

    # Filtrer umarkert sett
    treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                  umarkert_andel)

    # Treningssett
    treningssett = pd.concat([treningssett_markert, treningssett_umarkert],
                            axis=0)
    treningssett = treningssett.sort_index(ascending=True)


    # Antall splitt etter minste enxymtype
    n_splitt = n_splitt_enzym(rå_data, treningssett_markert)

    # Modelloptimalisering
    semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt,
                             dict_res=res_semi.dict_res_2_hel,
                             rå_data=rå_data,
                             treningssett_markert=treningssett_markert,
                             treningssett_umarkert=treningssett_umarkert,
                             treningssett=treningssett,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder_andel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

    # Lagring av resultater
    res_2_hel_semi = pd.DataFrame(res_semi.dict_res_2_hel)
    res_2_hel_opt_semi = pd.DataFrame(res_semi.dict_res_2_hel_opt)

    res_2_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_hel_semi.csv')
    res_2_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_hel_opt_semi.csv')

Beste modell

## Designproduksjon

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Setter treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

In [ ]:
# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt,
                         dict_res=res_semi.dict_res_1_design,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_1_design_semi = pd.DataFrame(res_semi.dict_res_1_design)
res_1_design_opt_semi = pd.DataFrame(res_semi.dict_res_1_design_opt)

res_1_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_design_semi.csv')
res_1_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_design_opt_semi.csv')

[I 2024-04-22 20:26:46,972] A new study created in memory with name: no-name-d4765988-5d3a-480e-bcdc-0d26f5cf70fa
[I 2024-04-22 20:39:08,606] Trial 0 finished with value: 773.5818846632824 and parameters: {'k1': 18, 'k2': 14, 'p1': 7, 'p2': 7, 'max_iters': 84, 'pool_size': 123}. Best is trial 0 with value: 773.5818846632824.
[I 2024-04-22 20:45:44,168] A new study created in memory with name: no-name-7789f484-46ad-434a-91d5-43b0ddf9a098
[I 2024-04-22 20:46:29,680] Trial 0 finished with value: 744.7789457467345 and parameters: {'n_estimators': 665, 'criterion': 'friedman_mse', 'max_depth': 28, 'min_samples_leaf': 2, 'max_features': 0.6373427447180475, 'maks_iterasjoner': 10, 'std_terskel': 0.029886293317025428}. Best is trial 0 with value: 744.7789457467345.
[I 2024-04-22 20:47:44,856] A new study created in memory with name: no-name-4b2e5912-0342-41d8-8d42-379377e7806c
[I 2024-04-22 20:48:16,205] Trial 0 finished with value: 1627.7980521794639 and parameters: {'k': 1069, 'graf_vektet_h

Beste modell

## Alternativ 1: Forskjellige markerte og umarkerte treningsandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert, _, _,\
      _, _, _\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert

  # Går igjennom andeler av markerte andeler
  for andel in andel_av_markert:

    # Finner tilsvarende umarkerte andeler
    umarkert_andel = andel_umarkert(andel, treningssett_markert, trening_umarkert)

    # Filtrer umarkert sett
    ubenyttet_andel = (1.0 - umarkert_andel)
    treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
    = train_test_split(trening_umarkert,
                       enzymtyper_umarkert,
                       test_size=ubenyttet_andel,
                       stratify=enzymtyper_umarkert,
                       random_state=random_seed)

    # Treningssett
    treningssett = pd.concat([treningssett_markert, treningssett_umarkert],
                            axis=0)
    treningssett = treningssett.sort_index(ascending=True)


    # Antall splitt etter minste enxymtype
    n_splitt = n_splitt_enzym(rå_data, treningssett_markert)


    # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
    semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt,
                             dict_res=res_semi.dict_res_1_design,
                             rå_data=rå_data,
                             treningssett_markert=treningssett_markert,
                             enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                             treningssett_umarkert=treningssett_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                             treningssett=treningssett,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_andel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

    # Lagring av resultater
    res_1_design_semi = pd.DataFrame(res_semi.dict_res_1_design)
    res_1_design_opt_semi = pd.DataFrame(res_semi.dict_res_1_design_opt)

    res_1_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_design_semi.csv')
    res_1_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_design_opt_semi.csv')

[I 2024-04-29 21:46:37,570] A new study created in memory with name: no-name-cb8ee96b-7bcd-465a-8650-b59cad48f96d
[I 2024-04-29 21:47:12,202] Trial 15 finished with value: 833.288358056867 and parameters: {'k1': 4, 'k2': 4, 'p1': 3, 'p2': 6, 'max_iters': 37, 'pool_size': 61}. Best is trial 8 with value: 752.7301227097283.
[I 2024-04-29 21:47:16,534] Trial 1 finished with value: 1263.2109948774064 and parameters: {'k1': 10, 'k2': 8, 'p1': 10, 'p2': 10, 'max_iters': 3, 'pool_size': 38}. Best is trial 1 with value: 1263.2109948774064.
[I 2024-04-29 21:47:36,890] Trial 0 finished with value: 764.2287255057835 and parameters: {'k1': 7, 'k2': 2, 'p1': 9, 'p2': 8, 'max_iters': 34, 'pool_size': 59}. Best is trial 0 with value: 764.2287255057835.
[I 2024-04-29 21:48:18,855] Trial 2 finished with value: 844.0651021437834 and parameters: {'k1': 3, 'k2': 9, 'p1': 9, 'p2': 10, 'max_iters': 30, 'pool_size': 93}. Best is trial 0 with value: 764.2287255057835.
[I 2024-04-29 21:48:24,206] Trial 3 finis

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Definerer treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

In [ ]:
# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt,
                         dict_res=res_semi.dict_res_2_design,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultatater
res_2_design_semi = pd.DataFrame(res_semi.dict_res_2_design)
res_2_design_opt_semi = pd.DataFrame(res_semi.dict_res_2_design_opt)

res_2_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_design_semi.csv')
res_2_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_design_opt_semi.csv')

## Alternativ 2: Forskjellige markerte treningssandeler og umarkerte andeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, _, _\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       test_ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert


  # Går igjennom andeler av markerte andeler
  for andel in andel_av_markert:

    # Finner tilsvarende umarkerte andeler
    umarkert_andel = andel_umarkert(andel, treningssett_markert, trening_umarkert)

    # Filtrer umarkert sett
    treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                  umarkert_andel)

    # Treningssett
    treningssett = pd.concat([treningssett_markert, treningssett_umarkert],
                            axis=0)
    treningssett = treningssett.sort_index(ascending=True)


    # Antall splitt etter minste enxymtype
    n_splitt = n_splitt_enzym(rå_data, treningssett_markert)

    #Modelloptimalisering
    semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt,
                          dict_res=res_semi.dict_res_2_design,
                          rå_data=rå_data,
                          treningssett_markert=treningssett_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          n_splitt=n_splitt,
                          n_runder=n_runder_andel,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)
    # Lagring av resultatater
    res_2_design_semi = pd.DataFrame(res_semi.dict_res_2_design)
    res_2_design_opt_semi = pd.DataFrame(res_semi.dict_res_2_design_opt)

    res_2_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_design_semi.csv')
    res_2_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_design_opt_semi.csv')


Beste modell

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

# Setter fast treningsandel

In [ ]:
# Definere andel av treningsett som skal benyttes
trenings_andel = 1.0

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt_NIR,
                         dict_res=res_semi.dict_res_1_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_1_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_NIR)
res_1_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_opt_NIR)

res_1_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_semi_NIR.csv')
res_1_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_opt_semi_NIR.csv')

Beste modell

## Alternativ 2: Manuell fordeling

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt_NIR,
                         dict_res=res_semi.dict_res_2_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_2_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_NIR)
res_2_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_opt_NIR)

res_2_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_hel_semi_NIR.csv')
res_2_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_hel_opt_semi_NIR.csv')

Beste modell

## Designproduksjon

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt_NIR,
                         dict_res=res_semi.dict_res_1_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_1_design_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_NIR)
res_1_design_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_opt_NIR)

res_1_design_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_design_semi_NIR.csv')
res_1_design_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_design_opt_semi_NIR.csv')

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt_NIR,
                         dict_res=res_semi.dict_res_2_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_2_design_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_NIR)
res_2_design_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_opt_NIR)

res_2_design_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_design_semi_NIR.csv')
res_2_design_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_2_design_opu_semi_NIR.csv')

Beste modell